In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplcursors
import seaborn as sns

#from prop13_utils.inflation_adjust import InflationAdj

%matplotlib inline
%matplotlib notebook

%load_ext autoreload
%autoreload 2

RAW_DATA_DIR = "../../../data/raw"
INTERIM_DATA_DIR = "../../../data/interim"

'''Note: The Respondent ID and Agency Code are the unique identifier for a reporter.
The Respondent ID, Agency Code, and Sequence Number combination make a unique record identifier. 
Each record is a mortgage application or a mortgate purchase.

glossary: https://www.ffiec.gov/hmda/glossary.htm
'''
COL_POS_MAP_90to03 = {
    "year": (0, 4),
    "respondent_id": (4, 14), # unique within agency
    "agency_code": (14, 15), # l=OCC, 2=FRS, 3=FDIC, 4=OTS , 5=NCUA, 7=HUD, B,C,D,E,or X=State Exempts
    "type_of_loan": (15, 16),
    "purpose_of_loan": (16, 17),
    "occupancy": (17, 18),
    "amount_of_loan": (18, 23),
    "type_of_action_taken": (23, 24), 
    "MSA": (24, 28),
    "state_code": (28, 30),
    "county_code": (30, 33),
    "census_tract_number": (33, 40),
    "applicant_race": (40, 41),
    "co-applicant_race": (41, 42),
    "applicant_sex": (42, 43),
    "co-applicant_sex": (43, 44),
    "applicant_income": (44, 48), # NA includes multifamily
    "type_of_purchaser": (48, 49),
    "denial_reason_1": (49, 50),
    "denial_reason_2": (50, 51),
    "denial_reason_3": (51, 52),
    "edit_status": (52, 53), 
    # blank = record has no edit failures, 
    # 5=Validity edit failure(s), 6=Quality edit failure(s), 7=Validity and Quality edit failures
    "seq_number": (53, 60), # A one up number assigned within a reporter, to make each record unique.
}

RACE_ORDER = ['AAPI', 'AI/AN', 'Black', 'Hispanic', 'White', 'other','not_provided', 'NA']

# None = not applicable
CODE_MAP = {
    "type_of_loan": {"1": "conventional", # any loan other than FHA, VA or FmHA loan
             "2": "FHA",
             "3": "VA",
             "4": "FmHA" # rural areas
            },
    "purpose_of_loan": {"1": "home_purchase_for1to4family",
                "2": "home_improv_for1to4family",
                "3": "refinance_for1to4family",
                "4": "multifamily"
               },
    'occupancy': {"1": "owner-occupied",
                  "2": "not_owner-occupied",
                  "3": None, #For multifamily dwellings, and for any dwellings located in MSAs where you do not have home or branch offices or located outside MSAs, you may either enter the code for "not applicable" or the code for the actual occupancy status.
                 },
    'type_of_action_taken': {"1": "originated",
                             "2": "approved_but_not_accepted", # not accpeted by app
                             "3": "denied", # including applicant refused a counteroffer
                             "4": "withdrawn_by_app",
                             "5": "closed_for_missing_info", # file closed for incompleteness,
                             "6": "purchased" # by institution
                            },
    "race": {"1": "AI/AN", #"American_Indian_or_Alaskan_Native"
            "2": "AAPI",
            "3": "Black",
            "4": "Hispanic",
            "5": "White",
            "6": "other",
            "7": "not_provided", # only for mail and phone apps as in-person based on name and appearance
            "8": None # purchased loan, institutions not required, applicant is not a natural person
            },
    "sex": {"1": "male", "2": "female", "3": "not_provided", "4": None # same as race
           },
    "agency_code": {"1": "OCC", 
                    # Office of the Comptroller of the Currency
                    "2": "FRS", 
                    "3": "FDIC",
                    "4": "OTS", 
                    "5": "NCUA",
                    "7": "HUD"},
    'edit_status': {" ": None,
                "5": "validity_edit_failure", # not correct
                "7": "validity_and_quality_edit_failures", # not correct
                "6": "quality_edit_failure", # not align with an expected standard 
               },
}

VALIDITY_FAILURE_EDIT_MAP = {"5": "validity_edit_failure", "7": "validity_and_quality_edit_failures"}

def get_valid_state_codes():
    df = pd.read_csv(f"{RAW_DATA_DIR}/HMDA/state_FIPS_code.txt", sep="|", dtype={"STATE": str})
    return list(df["STATE"]) + list(df["STUSAB"])

VALID_STATE_CODES = get_valid_state_codes()
CATEGORY_COLS = ['agency_code', 'type_of_loan', 'purpose_of_loan', 'occupancy', 'type_of_action_taken', 'edit_status',
                'applicant_race', 'co-applicant_race',
                 'applicant_sex', 'co-applicant_sex']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
def select_cols_and_interpret_code(df):
    cols = ['year', 'respondent_id', 'agency_code', 'type_of_loan',
       'purpose_of_loan', 'occupancy', 'amount_of_loan',
       'type_of_action_taken', 'MSA', 'state_code', 'county_code',
       'census_tract_number', 'applicant_race', 'co-applicant_race',
       'applicant_sex', 'co-applicant_sex', 'applicant_income', 'edit_status', 'seq_number']
    df = df[cols]
    
    for col in CATEGORY_COLS[:-4]:
        df[col] = df[col].map(lambda x: CODE_MAP[col].get(x, x))
    for app in ["applicant", "co-applicant"]:
        sex_col = app + "_sex"
        df[sex_col] = df[sex_col].map(lambda x: CODE_MAP["sex"].get(x, x))
        race_col = app + "_race"
        df[race_col] = df[race_col].map(lambda x: CODE_MAP["race"].get(x, x))
    
    return df

In [38]:
def plot_bar_chart_for_city_by_race(df):
    plt.figure(figsize=(12, 7))
    sns.set(style="darkgrid")
    
    ax = sns.barplot(
        data=df, 
        estimator=sum, 
        orient="h",
        color='#69b3a2',
    )
    return ax


In [ ]:
def read_data(file_path):
    with open(file_path, mode="r", encoding="ascii") as f:
        raw_txt = f.read()
    
    print("file read")
    lines = [raw_txt[i * 61: i * 61 + 61] for i in range(len(raw_txt) // 61)]
    lines = [l.rstrip("\n") for l in lines]
    print(len(lines))
    assert all(np.array([len(l) for l in lines]) == 60)
    print("length assert passed")
    df = []
    i=0
    for line in lines:
        _record = dict()
        i+=1
        for col, pos in COL_POS_MAP_90to03.items():
            _record[col] = line[pos[0]:pos[1]]
        df.append(_record)
        assert not("\r" in line or "\n" in line)
        print(i)
    print("data frame appended")
    df = pd.DataFrame.from_records(df)
    return df,lines

In [40]:
def validate_code(df):
    print("validating code")
    for col in CATEGORY_COLS:
        freq = df[col].value_counts(dropna=False)
        code_name = col
        if col.endswith("race"):
            code_name="race"
        if col.endswith("sex"):
            code_name="sex"
        for i in freq.index:
            if not i in CODE_MAP[code_name].keys():
                print(f"{col}: {freq[i]} with invalid code:{i}")

        cnt_not_audited_invalid = (~df.loc[~df[col].isin(CODE_MAP[code_name].keys()), "edit_status"].isin(VALIDITY_FAILURE_EDIT_MAP.keys())).sum()
        if cnt_not_audited_invalid > 0:
            print(f"{col} has {cnt_not_audited_invalid} invalid records not marked by edits")


In [41]:
def validate_state_code(df, df_ca):
    invalid_state = df.loc[
        ~df["state_code"].isin(VALID_STATE_CODES) &
        ((df["county_code"]!="NA ") |
        (df["MSA"]!="NA  ") | 
        (df["census_tract_number"]!="NA     ")), 
         ["seq_number","state_code", "county_code", "MSA", "census_tract_number", "edit_status"]]
    print(f"{invalid_state.shape[0]/df_ca.shape[0]} of CA records with invalid state but other area")
    print("unmarked cnt:", (~invalid_state["edit_status"].isin(VALIDITY_FAILURE_EDIT_MAP.keys())).sum())

In [42]:

def normalize_loans(df):
    msa_na_codes = ['NA  ',"0000",'na  ','    ','00-0', 'MSAN']
    county_na_codes = ['NA ', '000', '   ', 'N/A', 'N\\A', '0NA', 'N A']
    tract_na_codes = ['       ', 'N/A    ', 'NA    8', 'NONE   ', '00000NA', 'N A    ']
    
    df = df.copy()
    # in thousands
    df["amount_of_loan"] = df["amount_of_loan"].str.strip()
    df["amount_of_loan"] = df["amount_of_loan"].str.lstrip("0")
    df.loc[df["amount_of_loan"] == "", "amount_of_loan"] = "0.25"
    df["amount_of_loan"] = pd.to_numeric(df["amount_of_loan"], errors="raise") * 1000
    
    # one-to-four family
    df.loc[df["applicant_income"]=='    ', "applicant_income"] = 'NA  '
    df["applicant_income"] = df["applicant_income"].str.lstrip("0")
    df.loc[df["applicant_income"] == "", "applicant_income"] = "0.25" # 0000;<500
    df.loc[df["applicant_income"] == 'NA  ', "applicant_income"] = None
    df["applicant_income"] = pd.to_numeric(df["applicant_income"], errors="raise") * 1000
    
    df.loc[(df["MSA"]=='67.8') & (df['county_code']=="065"), "MSA"] = "6780"
    df["MSA"] = df["MSA"].map(lambda x: None if x in msa_na_codes else x)
    
    df.loc[df["county_code"]=="O37", "county_code"] = "037"
    df.loc[df["county_code"]=='LA ', "county_code"] = "037"
    df["county_code"] = df["county_code"].map(lambda x: None if x in county_na_codes else x)
    
    df["seq_number"] = df["seq_number"].str.lstrip().str.zfill(7)
        
    # not following census stract number standard are all validity edit failures
    df.loc[(df["census_tract_number"] == '96 04  ') & (df["county_code"] == "073") & (df["MSA"] == "7320"),
           "census_tract_number"] = "0096.04"
    df.loc[(df["census_tract_number"] == 'R992.25') & (df["county_code"] == "059") & (df["MSA"] == "0360"),
       "census_tract_number"] = "0992.25"
    df.loc[(df["census_tract_number"] == '1506.-2') & (df["county_code"] == "097") & (df["MSA"] == "7500"),
   "census_tract_number"] = "1506.02"
    df.loc[df["census_tract_number"].isin(tract_na_codes),"census_tract_number"] = 'NA     '
    df.loc[(~df["census_tract_number"].str.fullmatch(r"^\d{4}\.\d{2}$")) & (df["census_tract_number"] != 'NA     '),
           "census_tract_number"] =  'NA     '
    df.loc[df["census_tract_number"] == 'NA     ', "census_tract_number"] = None
    
    # impute applicant race
    df["applicant_race"] = np.where(
        (~df["applicant_race"].isin(CODE_MAP["race"].values())) & (df["co-applicant_race"].isin(CODE_MAP["race"].values())),
        df["co-applicant_race"],
        df["applicant_race"])
    
    # cols that cannot be matched with code map converted to NA
    for col in CATEGORY_COLS:
        code_name = col
        if col.endswith("race"):
            code_name="race"
        if col.endswith("sex"):
            code_name="sex"
        df.loc[~df[col].isin(CODE_MAP[code_name].values()), col] = None
    
    return df.reset_index(drop=True)

In [43]:
def get_home_purchase_loans_and_validation(data_ca):
    
    home_purchase_loans = data_ca[(data_ca["purpose_of_loan"] == "home_purchase_for1to4family") &
                                     (data_ca["type_of_action_taken"] == "originated")].copy()
    n_selected_loans = home_purchase_loans.shape[0]
    print("# originated 1-to-4family home purchase loans:", n_selected_loans)

    assert (home_purchase_loans.groupby(['respondent_id', 'agency_code', 'seq_number']).size() > 1).sum() ==0

    print("race:", home_purchase_loans["applicant_race"].unique())

    mask_app_race = home_purchase_loans["applicant_race"].isin(CODE_MAP["race"].values())
    n_imputed = home_purchase_loans[(~mask_app_race) & home_purchase_loans["co-applicant_race"].isin(CODE_MAP["race"].values())].shape[0]
    n_na = home_purchase_loans[~mask_app_race].shape[0]
    if n_na != 0:
        print(f"{n_imputed/n_na} of invalid race could be imputed from co-applicant")

    print(home_purchase_loans.loc[(~home_purchase_loans["applicant_income"].str.isdigit() & 
                                   (home_purchase_loans["applicant_income"]!='NA  ')), "applicant_income"].unique())

    invalid_app_income = home_purchase_loans.loc[(~home_purchase_loans["applicant_income"].str.isdigit() & 
                                                  (home_purchase_loans["applicant_income"]!='NA  ')), :]
    print("invliad app_income:", invalid_app_income.shape[0])
    print("# unmarked by edit", (~invalid_app_income["edit_status"].isin(VALIDITY_FAILURE_EDIT_MAP.values())).sum())
    print("income:", invalid_app_income)
    

    assert sum(~home_purchase_loans["amount_of_loan"].str.isdigit()) == 0
    invalid_loan_amount = home_purchase_loans.loc[home_purchase_loans["amount_of_loan"]=="00000", :]
    print("loan_amount:", invalid_loan_amount)


    invalid_tract = home_purchase_loans.loc[
        ~home_purchase_loans["census_tract_number"].str.fullmatch(r"^\d{4}\.\d{2}$") &
        (home_purchase_loans["census_tract_number"] != 'NA     ') &
        (~home_purchase_loans["census_tract_number"].isin(['       ', 'N/A    ', 'NA    8'])), 
        ["county_code", "census_tract_number", "MSA", 'respondent_id', 'agency_code', 'seq_number']]
    print("tract:")
    print(invalid_tract["census_tract_number"].unique())
    print(invalid_tract)

    print("MSA")
    print(home_purchase_loans.loc[~home_purchase_loans["MSA"].str.isdigit(),"MSA"].unique())
    print("county")
    print(home_purchase_loans.loc[~home_purchase_loans["county_code"].str.isdigit(),"county_code"].unique())
    
    return home_purchase_loans, n_selected_loans

In [44]:
def read_and_validate_data(yr, rec_cnt):
    data_file = f"{RAW_DATA_DIR}/HMDA/{yr}/HMS.U{yr}.LARS"
    data,lines = read_data(data_file)
    assert data.shape[0] == rec_cnt

    print(data.tail())

    assert len(data.year.unique()) == 1

    print(sorted(data["state_code"].unique()))

    ca_mask = (data["state_code"] == "06") | (data["state_code"] == "CA") | (data["state_code"]==' 6')
    data_ca = data[ca_mask].copy()

    validate_state_code(data, data_ca)

    # <1% of not marked invalid code
    validate_code(data_ca)

    data_ca = select_cols_and_interpret_code(data_ca)
    print("total loans #:", data_ca.shape[0])

    data_ca.to_pickle(f"{INTERIM_DATA_DIR}/hmda_{yr}_raw.pickle")
    return data_ca, lines

# 1998

In [45]:
yr = 1998
rec_cnt = 24701216

#data_98_ca, lines = read_and_validate_data(yr, rec_cnt)

In [46]:
#home_purchase_loans, n_selected_loans = get_home_purchase_loans_and_validation(data_98_ca)

In [47]:
#normalized_home_purchase_loans = normalize_loans(home_purchase_loans)
#n_normalized_home_purchase_loans = normalized_home_purchase_loans.shape[0]
#print("# normalized_home_purchase_loans", n_normalized_home_purchase_loans, n_normalized_home_purchase_loans/n_selected_loans)


#print("pct_valid_tract:", normalized_home_purchase_loans["census_tract_number"].str.fullmatch(r"^\d{4}\.\d{2}$").sum()/n_selected_loans)
#print("pct_na_tract:", normalized_home_purchase_loans["census_tract_number"].isna().sum()/n_selected_loans)


#hist_loans.append(normalized_home_purchase_loans)

In [48]:
#import pickle

#len(hist_loans)

#with open(f"{INTERIM_DATA_DIR}/hmda_90to98.pickle", "wb") as f:
#    pickle.dump(hist_loans, f)


# 1999

In [ ]:
yr = 1999
rec_cnt = 22911780

data_98_ca, lines = read_and_validate_data(yr, rec_cnt)

file read
22911425
length assert passed
